# Multinomial Logistic Regression 
This is an example of multinomial logistic regression using a data set from UCI's [machine learning repository](https://archive.ics.uci.edu/ml/datasets/student+performance#). The data set comes from UCI's machine learning repository and can be downloaded [here](https://archive.ics.uci.edu/ml/machine-learning-databases/00320/). A description of the data can be found [here](https://archive.ics.uci.edu/ml/datasets/student+performance).

## Import Data:
Import the data into a pandas dataframe. Get dummy variables for each categorical predictor in the data set and return the design matirx. Create a normalized and standardized design matrix as well to compare model preformance. Convert response variable to three classes *0 (fail) , 1 (pass),* and *2 (incomplete)*. A student is put into the passing class if they got a score bigger or equal to 10, the failing class if they got a score below 10 but above 0, and the incomplete calss if the student received a 0.

In [1]:
import time
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.utils.extmath import cartesian
from sklearn import metrics
from sklearn import preprocessing

df = pd.read_csv('student-por2.csv')
df = pd.get_dummies(df, drop_first=True)

## Correlation Matrix
Next, we review the correlation matrix to see which variables are highly correlated. Assesing this metric will help us identify variables that potentially will inflate the variance and thus should be removed from the model. **Note** If you wish to view the full correlation matrix, remove the comment from the below syntax and open *corr_matrix.csv* in your current working directory.

In [2]:
corr_mat = df.corr(method = 'pearson')
#corr_mat.to_csv("corr_matrix_pear.csv")
corr_mat

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
age,1.000000,-0.107832,-0.121050,0.034490,-0.008415,0.319968,-0.020559,-0.004910,0.112805,0.134768,...,-0.048726,0.330353,-0.167841,-0.101894,-0.005458,-0.054279,-0.021441,-0.265497,0.013115,0.178810
Medu,-0.107832,1.000000,0.647477,-0.265079,0.097006,-0.172210,0.024421,-0.019686,0.009536,-0.007018,...,0.091562,-0.101123,-0.022168,0.120491,0.113973,0.119354,0.125951,0.213896,0.266052,-0.030992
Fedu,-0.121050,0.647477,1.000000,-0.208288,0.050400,-0.165915,0.020256,0.006841,0.027690,0.000061,...,-0.044450,-0.066684,0.023572,0.135191,0.094628,0.079700,0.074863,0.191735,0.183483,-0.067675
traveltime,0.034490,-0.265079,-0.208288,1.000000,-0.063154,0.097730,-0.009521,0.000937,0.057454,0.092824,...,-0.066130,0.090497,-0.044807,-0.039289,-0.044842,-0.033376,-0.011509,-0.071958,-0.190826,0.004751
studytime,-0.008415,0.097006,0.050400,-0.063154,1.000000,-0.147441,-0.004127,-0.068829,-0.075442,-0.137585,...,-0.018076,0.006440,0.089316,0.143509,-0.002314,0.070080,0.042630,0.188256,0.037529,0.033036
failures,0.319968,-0.172210,-0.165915,0.097730,-0.147441,1.000000,-0.062645,0.108995,0.045078,0.105949,...,-0.056527,0.234027,-0.000745,-0.006982,0.069416,0.000561,-0.069241,-0.309400,-0.095330,0.069901
famrel,-0.020559,0.024421,0.020256,-0.009521,-0.004127,-0.062645,1.000000,0.129216,0.089707,-0.075767,...,0.012507,-0.067365,-0.012038,0.015228,0.031937,0.057597,0.041055,0.048239,0.082214,-0.044920
freetime,-0.004910,-0.019686,0.006841,0.000937,-0.068829,0.108995,0.129216,1.000000,0.346352,0.109904,...,0.022349,0.033823,-0.015611,0.003764,-0.049574,0.150329,-0.007096,-0.102618,0.063268,0.027112
goout,0.112805,0.009536,0.027690,0.057454,-0.075442,0.045078,0.089707,0.346352,1.000000,0.245126,...,0.042602,0.018432,-0.058124,0.017262,-0.006683,0.088582,0.018679,-0.069105,0.092869,-0.000520
Dalc,0.134768,-0.007018,0.000061,0.092824,-0.137585,0.105949,-0.075767,0.109904,0.245126,1.000000,...,-0.093064,0.112437,-0.028076,-0.016844,0.051986,0.022592,-0.078376,-0.131663,0.042811,0.062042


## Variance Inflation Factor
Analyzing the correlation between variables is not enough. It is very beneficial to access which variables inflate the variance the most. Since models with minimum variance are optimal we will remove variables that inflate variance the most. In general a VIF bigger than or equal to 10 is considered high VIF and thus, the associated variable should be removed from the design matrix.

In [3]:
def var_if_fac(data_frame, ind_var):
    index = data_frame.columns.get_loc(ind_var)
    mat = data_frame.as_matrix()
    return(vif(mat, index))

no_response = df.drop('G3',1)
arr1 = []
arr2 = list(no_response)

for i in list(no_response):
    arr1.append(var_if_fac(no_response,i))
vif_df = pd.DataFrame(list(zip(arr2,arr1)),columns = ['Ind_Var','VIF'])
vif_df

,Ind_Var,VIF
0,age,92.485829
1,Medu,15.909121
2,Fedu,11.305630
3,traveltime,6.747138
4,studytime,8.042191
5,failures,1.620601
6,famrel,19.965318
7,freetime,12.451270
8,goout,12.117377
9,Dalc,6.680433


# Remove Correlated Variables/Transform Response Variable
The variables *G1*, *G2*, and *G3* had the higest correlation and thus we will exclude *G1* and *G2* from the design matrix. We keep *G3* becasue that is our response variable. We will also create our multidemonsional design matrix and 1-demensional response variable in this step. Lastly we start with defining a function *response_conv()* to transform our diescrete response variable to binary.

In [4]:
def response_conv(arr):
    new = []
    for i in arr:
        if (i > 0 and i < 10):           # condition where student failed
            new.append(0)                 
                                          
        elif (i >= 10):                   # condition where student passed
            new.append(1)                 
    
        else:                             # condition where student received an incomplete
            new.append(2)
    return(new)                           # 1-dimensional response varibale returned

drop_col_names = []

vifs = list(vif_df.VIF)
predictors = list(vif_df.Ind_Var)

for i in range(len(predictors)):
    if vifs[i] >= 10:
        drop_col_names.append(predictors[i])
        
df = df.drop(drop_col_names,1)            # this is the data frame with high VIF variables removed
X = df.drop('G3',1)                       # this is the design matrix
y = list(df.G3)                           # this is the discrete response vector
y_new = response_conv(y)                  # this is the multinomial response vector
X_scale = preprocessing.scale(X)
X_norm = preprocessing.normalize(X)

## Niave Accuracy/Split Data
Before we start training and selecting parametrs for our model, we must find the distribution of the classes amongst the response variable. Depnding on which class is the dominate class, our model should preform better than just guessing the dominate class for each observation. For example, if the dominate class is 1 and 1's comprise of 83% of the response data, then our model should have higher than 83% accuracy. 

In [5]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y_new, test_size=0.33, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X_scale, y_new, test_size=0.33, random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(X_norm, y_new, test_size=0.33, random_state=42)

zero = 0
one = 0
two = 0

for i in y1_test:
    if i == 0:
        zero += 1
    elif i == 1:
        one += 1
    else:
        two += 1

num1 = round((zero/len(y1_test))*100,2)
num2 = round((one/len(y1_test))*100,2)
num3 = round((two/len(y1_test))*100,2)
print("The response vector has the following distribution: \nzeros: %r zeros comprising of %r percent of the response data. \nones: %r ones comprising of %r percent of the response data. \ntwos: %r twos comprising of %r percent of the response data." % (zero,num1,one,num2,two,num3))
print("\n")

The response vector has the following distribution: 
zeros: 23 zeros comprising of 10.7 percent of the response data. 
ones: 187 ones comprising of 86.98 percent of the response data. 
twos: 5 twos comprising of 2.33 percent of the response data.




## Fit Model
Fit the logistic regression model and perform cross validation.

In [6]:
log_reg1 = LogisticRegressionCV(cv=10,scoring='neg_log_loss',random_state=42).fit(X1_train, y1_train)
log_reg2 = LogisticRegressionCV(cv=10,scoring='neg_log_loss',random_state=42).fit(X2_train, y2_train)
log_reg3 = LogisticRegressionCV(cv=10,scoring='neg_log_loss',random_state=42).fit(X3_train, y3_train)

## Estimated Coefficients
Due to the multinomial nature of the response variable, each estimated coefficient is displayed in a vector consisting of 3 components, one for each response category. The follwing output captures this notion.

In [7]:
coef_df = pd.DataFrame(list(zip(X1_train.columns, np.transpose(np.round(log_reg1.coef_,2)))))
coef_df

,0,1
0,traveltime,"[0.02, 0.07, 0.22]"
1,studytime,"[-0.13, 0.23, -0.28]"
2,failures,"[0.54, -0.96, 0.79]"
3,Dalc,"[0.09, -0.14, 0.22]"
4,Walc,"[0.1, -0.06, 0.0]"
5,health,"[0.01, -0.03, 0.06]"
6,absences,"[0.07, -0.06, -0.88]"
7,school_MS,"[0.52, -1.45, 0.85]"
8,sex_M,"[0.03, -0.15, 0.09]"
9,address_U,"[-0.07, 0.2, -0.49]"


## Probability Per Category
Each observation is assigned a probability for each respective response category. In our case each observation will then have 3 probabilities, one for student failing (0), one for student passing (1), and one for student receiving an incomplete (2). The following table is a snipit of each observations probability for each respective category. **Note** each row sums to 1.

In [8]:
pred_prob = log_reg1.predict_proba(X1_test)
prob_df = pd.DataFrame(np.round(pred_prob,4), columns = ['prob_fail','prob_pass', 'prob_inc'])
prob_df.index.name = 'Obs'
prob_df


,prob_fail,prob_pass,prob_inc
Obs,,,
0,0.1012,0.8969,0.0019
1,0.0892,0.9091,0.0017
2,0.0635,0.8853,0.0512
3,0.1892,0.8104,0.0004
4,0.0958,0.9004,0.0038
5,0.1337,0.8658,0.0005
6,0.0585,0.9262,0.0153
7,0.4959,0.5041,0.0000
8,0.1334,0.8666,0.0000


## Model Prediction vs. Actual Response Value
The following is the comparrison of the predictions made by our model (y_pred) and the actual response values (y_act), on the testing data set.

In [9]:
log_pred1 = log_reg1.predict(X1_test)
log_pred2 = log_reg2.predict(X2_test)
log_pred3 = log_reg3.predict(X3_test)

pred = pd.DataFrame(list(zip(y1_test, log_pred1, log_pred2, log_pred3)), columns=['y_act','y_log','y_stan_log','y_norm_log'])
pred.index.name = 'Obs'

#pred.to_csv("preds.csv")
pred

,y_act,y_log,y_stan_log,y_norm_log
Obs,,,,
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1
5,1,1,1,1
6,1,1,1,1
7,0,1,1,1
8,1,1,1,1


## Results
Accuracy, confusion matrix, and classification reports are returned for the standardized design matirx. **Note** We don't have a model that beats niave accuracy, thus we won't be selecting any of these models as our final model.

In [10]:
cm_lr1 = pd.DataFrame(metrics.confusion_matrix(y1_test, log_pred1), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])
cm_lr2 = pd.DataFrame(metrics.confusion_matrix(y2_test, log_pred2), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])
cm_lr3 = pd.DataFrame(metrics.confusion_matrix(y3_test, log_pred3), index = ['Fail(0)','Pass(1)','Inc(2)'],columns=['Fail(0)','Pass(1)','Inc(2)'])


print ("The accuracy of the Non-standardized Logistic Regression model is: ", log_reg1.score(X1_test,y1_test))
print ("\n")
print ("The accuracy of the Standardized Logistic Regression model is: ", log_reg2.score(X2_test,y2_test))
print ("\n")
print ("The accuracy of the Normalized Logistic Regression model is: ", log_reg3.score(X3_test,y3_test))
print ("\n")

print("Non-standardized Logistic Regression Confusion Matrix: \n", cm_lr1)
print ("\n")
print("Standardized Logistic Regression Confusion Matrix: \n", cm_lr2)
print ("\n")
print("Normalized Logistic Regression Confusion Matrix: \n", cm_lr3)
print ("\n")

print("Classification report for Non-standardized design matrix:\n", metrics.classification_report(y1_test,log_pred1))
print("\n")
print("Classification report for standardized design matrix:\n", metrics.classification_report(y2_test,log_pred2))
print("\n")
print("Classification report for Normalized design matrix:\n", metrics.classification_report(y3_test,log_pred3))

The accuracy of the Non-standardized Logistic Regression model is:  0.874418604651


The accuracy of the Standardized Logistic Regression model is:  0.874418604651


The accuracy of the Normalized Logistic Regression model is:  0.86976744186


Non-standardized Logistic Regression Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)        3       20       0
Pass(1)        2      185       0
Inc(2)         0        5       0


Standardized Logistic Regression Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)        3       20       0
Pass(1)        2      185       0
Inc(2)         0        5       0


Normalized Logistic Regression Confusion Matrix: 
          Fail(0)  Pass(1)  Inc(2)
Fail(0)        2       21       0
Pass(1)        3      184       0
Inc(2)         0        4       1


Classification report for Non-standardized design matrix:
              precision    recall  f1-score   support

          0       0.60      0.13      0.21        23
          1    

C:\Users\006988889-SA\Downloads\conda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
